In [1]:
import shutil

try:
    shutil.rmtree(r"C:\last_project\make_domain_selector\table_info")
except:
    pass

In [2]:
from sentence_transformers import SentenceTransformer
import chromadb

c:\last_project\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import json

table_info = []
table_metadata = []

with open(r"C:\last_project\make_domain_selector\mimic_table_info_llm_llm_optimized.json", "r", encoding="utf-8") as f:
    mimic_table_info = json.load(f)

In [4]:
for domain in ['hosp', 'icu']:
    for table_name in mimic_table_info[domain].keys():

        table_inst = mimic_table_info[domain][table_name]['table_instruction'].strip()
        columns_text = ''

        for col_name, col_info in mimic_table_info[domain][table_name]['columns_instruction'].items():
            if 'column_description' in col_info:

                short_desc = col_info['column_description'].replace('\n',' ').split('.')[0].strip()
                
                columns_text += (
                    f"- {col_name} ({col_info['data_type']}): "
                    f"{short_desc}.\n"
                )

        all_table_info = f"""
The table {table_name} in the MIMIC-IV database.

Key column descriptions:
{columns_text}
""".strip()

        table_info.append(all_table_info)
        table_metadata.append({'table_name' : table_name})

# {table_inst}

In [5]:
embedding_model = SentenceTransformer("intfloat/e5-small-v2")

def embed_passages(texts):
    texts = [f"passage: {t}" for t in texts]
    return embedding_model.encode(texts, batch_size=64, show_progress_bar=True).tolist()

client = chromadb.PersistentClient(
    path=r"C:\last_project\make_domain_selector\\table_info"
)

# 기존 컬렉션 삭제
try:
    client.delete_collection("table_info")
except:
    pass

collection = client.create_collection(name="table_info")

embeddings = embed_passages(table_info)

collection.add(
    documents=table_info,
    embeddings=embed_passages(table_info),
    metadatas=table_metadata,
    ids= [f'table_{i}' for i in range(len(table_info))])

Batches: 100%|██████████| 1/1 [00:00<00:00,  1.78it/s]
